In [ ]:
# 安裝 Playwright 與 gspread-formatting（第一次跑時要執行）
!pip install -q playwright gspread-formatting
!playwright install chromium

import nest_asyncio
import asyncio
from playwright.async_api import async_playwright

nest_asyncio.apply()

async def scrape_pigepm():
    async with async_playwright() as p:
        browser = await p.chromium.launch(headless=True, args=["--no-sandbox"])
        page = await browser.new_page()
        await page.goto("https://pigepm.moa.gov.tw/", wait_until="networkidle")

        items = await page.query_selector_all('.number-name-item')
        farm_count = user_count = None

        for item in items:
            label = (await item.inner_text()).strip()
            value = (await item.evaluate("el => el.previousElementSibling.textContent")).strip()
            if '牧場數量' in label:
                farm_count = int(value)
            elif '使用者數量' in label:
                user_count = int(value)

        await browser.close()
        return farm_count, user_count

farm, user = await scrape_pigepm()

print("🐷 牧場數量：", farm)
print("👥 使用者數量：", user)

# ====== 寫入 Google Sheet ======
import gspread
import google.auth
from google.colab import auth
from datetime import datetime
from gspread_formatting import *

auth.authenticate_user()
creds, _ = google.auth.default()
gc = gspread.authorize(creds)

SHEET_ID = "1BRfNr84btjJFPH9CXUiTMeojHGb16Y7vRf_D92kHKOU"
WORKSHEET_NAME = "數據記錄"

sh = gc.open_by_key(SHEET_ID)
worksheet = sh.worksheet(WORKSHEET_NAME)

# 如果還沒標題，先加上
if worksheet.cell(1, 1).value != "日期時間":
    worksheet.insert_row(["日期時間", "牧場數量", "使用者數量", "數據來源"], 1)

# 寫入資料（使用 datetime 字串）
timestamp = datetime.now().strftime("%Y/%m/%d %H:%M:%S")
row = [timestamp, farm, user, "Playwright-Colab"]

print("✅ 準備寫入資料中...")
worksheet.append_row(row, value_input_option='USER_ENTERED')
print("✅ 寫入完成")

# 格式化 A 欄為 yyyy/MM/dd HH:mm:ss
fmt = cellFormat(numberFormat=numberFormat(type='DATE_TIME', pattern='yyyy/MM/dd HH:mm:ss'))
format_cell_range(worksheet, 'A:A', fmt)

print("✅ 已成功寫入 Google Sheet！")


🐷 牧場數量： 986
👥 使用者數量： 1063
✅ 準備寫入資料中...
✅ 寫入完成
✅ 已成功寫入 Google Sheet！
